In [ ]:
from sqlalchemy import create_engine, inspect
import pandas as pd
import numpy as np

# Configuration de la connexion
username = 'mia17-hackaton'
password = 'hackaton-mia17'
host = 'postgresql-mia17-hackaton.alwaysdata.net'
port = 5432
database = 'mia17-hackaton_psql'

# URL de connexion
database_url = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Créer une engine de connexion
engine = create_engine(database_url)

# Connexion à la base de données
connection = engine.connect()

# Récupérer les noms des tables
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables disponibles dans la base de données :", tables)

Tables disponibles dans la base de données : ['cleaned_olympic_data', 'cleaned_olympic_results', 'olympic_athletes', 'olympic_hosts', 'cleaned_olympic_results1', 'olympic_medals', 'cleaned_olympic_results1_cleaned', 'olympic_results']


In [ ]:
df_cleaned_olympic_results1 = pd.read_sql_table('cleaned_olympic_results1', engine)
df_cleaned_olympic_results1

,_id,discipline_title,event_title,slug_game,participant_type,medal_type,country_name,athlete_full_name,value_type,games_participations,first_game,gold_medals,silver_medals,bronze_medals,total_medals
0,162113,Fencing,foil individual men,athens-1896,Athlete,BRONZE,Greece,Periklis Pierrakos-Mavromikhalis,None,2.0,Unknown,0,0,0,0
1,162114,Fencing,foil individual men,athens-1896,Athlete,None,Greece,Konstantinos KOMNINOS-MILIOTIS,None,1.0,Athens 1896,0,0,0,0
2,162115,Fencing,foil individual men,athens-1896,Athlete,None,France,Henri DE LABORDE,None,2.0,Athens 1896,0,0,0,0
3,162116,Fencing,foil individual men,athens-1896,Athlete,None,Greece,Ioannis POULOS,None,1.0,Athens 1896,0,0,0,0
4,162117,Fencing,foil individual men,athens-1896,Athlete,None,Greece,Georgios Valakakis,None,2.0,Unknown,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164425,27098,Canoe Slalom,K1 kayak single women,london-2012,Athlete,None,Slovenia,Eva TERCELJ,None,2.0,London 2012,0,0,0,0
164426,27099,Canoe Slalom,K1 kayak single women,london-2012,Athlete,None,New Zealand,Luuka JONES,None,4.0,Beijing 2008,0,1,0,1
164427,27100,Canoe Slalom,K1 kayak single women,london-2012,Athlete,None,Italy,Maria Clara GIAI PRON,None,1.0,London 2012,0,0,0,0
164428,27101,Canoe Slalom,K1 kayak single women,london-2012,Athlete,None,Brazil,Ana SATILA,None,3.0,London 2012,0,0,0,0


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Charger les données
df = pd.read_sql_table('cleaned_olympic_results1', engine)

# Afficher les informations initiales
print(df.info())

# Remplacer les valeurs manquantes dans 'medal_type' par 'None'
df['medal_type'].fillna('None', inplace=True)

# Remplacer les valeurs manquantes dans 'first_game' par 'Unknown'
df['first_game'].fillna('Unknown', inplace=True)

# Remplacer les valeurs manquantes dans 'games_participations' par la moyenne
df['games_participations'].fillna(df['games_participations'].mean(), inplace=True)

# Convertir les colonnes en minuscules et supprimer les espaces
df['discipline_title'] = df['discipline_title'].str.lower().str.replace(' ', '_')
df['event_title'] = df['event_title'].str.lower().str.replace(' ', '_')
df['slug_game'] = df['slug_game'].str.lower().str.replace(' ', '_')
df['country_name'] = df['country_name'].str.lower().str.replace(' ', '_')
df['athlete_full_name'] = df['athlete_full_name'].str.lower().str.replace(' ', '_')

# Encodage des variables catégorielles
label_encoders = {}
categorical_columns = ['discipline_title', 'event_title', 'slug_game', 'participant_type', 'medal_type', 'country_name', 'athlete_full_name', 'first_game']

for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Normaliser les colonnes numériques (optionnel)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['games_participations', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']] = scaler.fit_transform(df[['games_participations', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']])

# Afficher les informations finales
print(df.info())
print(df.head())

# Sauvegarder le dataframe nettoyé
df.to_sql('cleaned_olympic_results1_cleaned', engine, if_exists='replace')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164430 entries, 0 to 164429
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   164430 non-null  int64  
 1   discipline_title      164430 non-null  object 
 2   event_title           164430 non-null  object 
 3   slug_game             164430 non-null  object 
 4   participant_type      164430 non-null  object 
 5   medal_type            20371 non-null   object 
 6   country_name          164430 non-null  object 
 7   athlete_full_name     143292 non-null  object 
 8   value_type            90945 non-null   object 
 9   games_participations  164430 non-null  float64
 10  first_game            164430 non-null  object 
 11  gold_medals           164430 non-null  int64  
 12  silver_medals         164430 non-null  int64  
 13  bronze_medals         164430 non-null  int64  
 14  total_medals          164430 non-null  int64  
dtype

430

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np

# Charger les données
df = pd.read_sql_table('cleaned_olympic_results1_cleaned', engine)

# Remplacer les valeurs manquantes dans 'value_type' par une valeur par défaut
df['value_type'].fillna('None', inplace=True)

# Convertir les colonnes catégorielles en type 'str'
df = df.astype({
    'discipline_title': 'str',
    'event_title': 'str',
    'slug_game': 'str',
    'participant_type': 'str',
    'medal_type': 'str',
    'country_name': 'str',
    'athlete_full_name': 'str',
    'value_type': 'str',
    'first_game': 'str'
})

# Encodage des variables catégorielles
label_encoders = {}
categorical_columns = ['discipline_title', 'event_title', 'slug_game', 'participant_type', 'medal_type', 'country_name', 'athlete_full_name', 'value_type', 'first_game']

for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Sélectionner les caractéristiques et la cible
X = df.drop(['medal_type', '_id'], axis=1)  # Ne pas utiliser 'medal_type' et '_id' comme caractéristiques
y = df['medal_type']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir les données en numpy arrays et changer les types
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

# Normaliser les caractéristiques
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Créer le modèle
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/50
4111/4111 [==============================] - 22s 4ms/step - loss: -229942032.0000 - accuracy: 0.0406 - val_loss: -994902400.0000 - val_accuracy: 0.0397
Epoch 2/50
4111/4111 [==============================] - 17s 4ms/step - loss: -4085624064.0000 - accuracy: 0.0406 - val_loss: -8976736256.0000 - val_accuracy: 0.0397
Epoch 3/50
4111/4111 [==============================] - 19s 5ms/step - loss: -19093512192.0000 - accuracy: 0.0406 - val_loss: -32539770880.0000 - val_accuracy: 0.0397
Epoch 4/50
4111/4111 [==============================] - 19s 5ms/step - loss: -54894448640.0000 - accuracy: 0.0406 - val_loss: -82501246976.0000 - val_accuracy: 0.0397
Epoch 5/50
4111/4111 [==============================] - 18s 4ms/step - loss: -123618975744.0000 - accuracy: 0.0406 - val_loss: -172070846464.0000 - val_accuracy: 0.0397
Epoch 6/50
4111/4111 [==============================] - 20s 5ms/step - loss: -239607431168.0000 - accuracy: 0.0406 - val_loss: -316717793280.0000 - val_accuracy: 0.0397


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Charger les données
df = pd.read_sql_table('cleaned_olympic_results1_cleaned', engine)

# Remplacer les valeurs manquantes dans 'value_type' par une valeur par défaut
df['value_type'].fillna('None', inplace=True)

# Convertir les colonnes catégorielles en type 'str'
df = df.astype({
    'discipline_title': 'str',
    'event_title': 'str',
    'slug_game': 'str',
    'participant_type': 'str',
    'medal_type': 'str',
    'country_name': 'str',
    'athlete_full_name': 'str',
    'value_type': 'str',
    'first_game': 'str'
})

# Encodage des variables catégorielles
label_encoders = {}
categorical_columns = ['discipline_title', 'event_title', 'slug_game', 'participant_type', 'country_name', 'athlete_full_name', 'value_type', 'first_game']

for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Encodage de 'medal_type'
medal_type_encoder = LabelEncoder()
df['medal_type'] = medal_type_encoder.fit_transform(df['medal_type'])

# Vérifier les classes de 'medal_type'
print(df['medal_type'].value_counts())

# Sélectionner les caractéristiques et la cible
X = df.drop(['medal_type', '_id'], axis=1)  # Ne pas utiliser 'medal_type' et '_id' comme caractéristiques
y = df['medal_type']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir les données en numpy arrays et changer les types
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

# Normaliser les caractéristiques
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Vérifier le nombre de classes dans 'medal_type'
num_classes = len(np.unique(y))

# Adapter la fonction de perte en fonction du nombre de classes
if num_classes == 2:
    loss_function = 'binary_crossentropy'
    activation_function = 'sigmoid'
else:
    loss_function = 'sparse_categorical_crossentropy'
    activation_function = 'softmax'

# Créer le modèle
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation=activation_function))

# Compiler le modèle
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])

# Définir les callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entraîner le modèle
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


medal_type
2    144059
0      7086
1      6653
3      6632
Name: count, dtype: int64
Epoch 1/100
4111/4111 [==============================] - 20s 4ms/step - loss: 0.3674 - accuracy: 0.8744 - val_loss: 0.3228 - val_accuracy: 0.8795
Epoch 2/100
4111/4111 [==============================] - 16s 4ms/step - loss: 0.3347 - accuracy: 0.8784 - val_loss: 0.3206 - val_accuracy: 0.8820
Epoch 3/100
4111/4111 [==============================] - 16s 4ms/step - loss: 0.3288 - accuracy: 0.8791 - val_loss: 0.3165 - val_accuracy: 0.8807
Epoch 4/100
4111/4111 [==============================] - 18s 4ms/step - loss: 0.3262 - accuracy: 0.8793 - val_loss: 0.3175 - val_accuracy: 0.8791
Epoch 5/100
4111/4111 [==============================] - 17s 4ms/step - loss: 0.3256 - accuracy: 0.8800 - val_loss: 0.3146 - val_accuracy: 0.8829
Epoch 6/100
4111/4111 [==============================] - 16s 4ms/step - loss: 0.3245 - accuracy: 0.8805 - val_loss: 0.3145 - val_accuracy: 0.8827
Epoch 7/100
4111/4111 [================

In [ ]:
import pandas as pd
import numpy as np

# Prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Si c'est une classification binaire
if num_classes == 2:
    predictions = np.round(predictions).astype(int).flatten()
else:
    predictions = np.argmax(predictions, axis=1)

# Décoder les prédictions
decoded_predictions = medal_type_encoder.inverse_transform(predictions)

# Ajouter les prédictions au dataframe de test
X_test_df = pd.DataFrame(X_test, columns=X.columns)
X_test_df['predicted_medal_type'] = decoded_predictions
X_test_df['country_name'] = df['country_name'].iloc[X_test_df.index].values

# Calculer le nombre de médailles par pays
medal_counts = X_test_df.groupby('country_name')['predicted_medal_type'].value_counts().unstack(fill_value=0)

# Renommer les colonnes pour plus de clarté
medal_counts.columns = ['No Medal', 'Bronze', 'Gold', 'Silver']

# Ajouter une colonne pour le total des médailles
medal_counts['Total'] = medal_counts.sum(axis=1)

# Trier par le total des médailles décroissant
medal_counts = medal_counts.sort_values(by='Total', ascending=False)

# Afficher le tableau des médailles par pays
print(medal_counts)

# Optionnel: Enregistrer le tableau des médailles dans un fichier CSV
#edal_counts.to_csv('medal_counts_by_country.csv')


1028/1028 [==============================] - 4s 4ms/step
              No Medal  Bronze  Gold  Silver  Total
country_name                                       
165                  6      37  1930      18   1991
6                    9      23  1238       8   1278
261                 11      20  1220       6   1257
30                   8      15  1221      11   1255
221                  6      25  1141      14   1186
...                ...     ...   ...     ...    ...
43                   0       0     1       0      1
146                  0       0     1       0      1
38                   0       0     1       0      1
174                  0       0     1       0      1
229                  0       0     0       1      1

[222 rows x 5 columns]
